In [2]:
!porder bundles --item "PSScene4Band"

Bundle type: all
basic_udm, analytic_dn, analytic_dn_xml, udm, analytic, analytic_xml, basic_analytic, basic_analytic_rpc, basic_analytic_xml, basic_analytic_dn, basic_analytic_dn_xml, analytic_sr

Bundle type: all_udm2
basic_udm, basic_udm2, analytic_dn, analytic_dn_xml, udm, udm2, analytic, analytic_xml, basic_analytic, basic_analytic_rpc, basic_analytic_xml, basic_analytic_dn, basic_analytic_dn_xml, analytic_sr

Bundle type: analytic
analytic, analytic_xml, udm

Bundle type: analytic_sr
analytic_sr, udm, analytic_xml

Bundle type: analytic_sr_udm2
analytic_sr, analytic_xml, udm, udm2

Bundle type: analytic_udm2
analytic, analytic_xml, udm, udm2

Bundle type: basic_analytic
basic_analytic, basic_udm, basic_analytic_rpc, basic_analytic_xml

Bundle type: basic_analytic_nitf
basic_analytic_nitf, basic_analytic_rpc_nitf, basic_analytic_xml_nitf, basic_udm

Bundle type: basic_analytic_nitf_udm2
basic_analytic_nitf, basic_analytic_rpc_nitf, basic_analytic_xml_nitf, basic_udm, basic_udm2

B